<a href="https://colab.research.google.com/gist/qbeer/22fe5333a1bd5c329fc2982d7dc5f7e0/lab03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised learning introduction, K-Nearest Neighbors (KNN)

Your task will be to predict wine quality from physicochemical features with the help of the 
[Wine Quality Data Set](https://archive.ics.uci.edu/ml/datasets/Wine+Quality). You will have to do it both as a regression and classification task. 


-------

### 1. Read data
  - Read the provided winequality-red.csv file. 
  - Check for missing values and that all entries are numerical. Also, check for duplicated entries (rows) and drop them.  
  - Use all columns except the last as features and the quality column as target. 
  - Make 80-20% train test split (use sklearn).
  - Prepare a one-hot encoded version of the y_test and y_train values ie. make a six long vector of the 6 quality classes (3-8), with only one non-zero value, e.g. 3->[1,0,0,0,0,0], 4->[0,1,0,0,0,0], 5->[0,0,1,0,0,0] etc. (You can use pandas or sklearn for that.) *You will have to use the one-hot encoded labels in the classification exercise only.*
  - Normalize the features by substracting the means and dividing by the standard deviation feature by feature. If you want to be very precise, you should use only the mean and std in the training set for normalization, because generally the test test is not available at training time.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
wines = pd.read_csv('winequality-red.csv', sep=';')
wines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
fixed acidity           1599 non-null float64
volatile acidity        1599 non-null float64
citric acid             1599 non-null float64
residual sugar          1599 non-null float64
chlorides               1599 non-null float64
free sulfur dioxide     1599 non-null float64
total sulfur dioxide    1599 non-null float64
density                 1599 non-null float64
pH                      1599 non-null float64
sulphates               1599 non-null float64
alcohol                 1599 non-null float64
quality                 1599 non-null int64
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [8]:
print('Are there NaN values? ' + str(np.any(wines.isna())))
print('Duplications: ' + str(wines.duplicated().sum()))

Are there NaN values? False
Duplications: 240


In [9]:
wines = wines[~wines.duplicated()].reset_index(drop=True)
print(wines.duplicated().sum())

0


In [13]:
from sklearn.model_selection import train_test_split #to split up the data into training and testing
feature = wines.columns[0:11] #select the features
X_train, X_test, y_train, y_test = train_test_split(wines[feature],
                 wines['quality'],
                 train_size=0.8)

In [14]:
X_train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
1311,11.1,0.440,0.42,2.2,0.064,14.0,19.0,0.99758,3.25,0.57,10.4
1132,8.4,0.390,0.10,1.7,0.075,6.0,25.0,0.99581,3.09,0.43,9.7
178,8.8,0.370,0.48,2.1,0.097,39.0,145.0,0.99750,3.04,1.03,9.3
1009,10.2,0.400,0.40,2.5,0.068,41.0,54.0,0.99754,3.38,0.86,10.5
595,7.7,0.660,0.04,1.6,0.039,4.0,9.0,0.99620,3.40,0.47,9.4
...,...,...,...,...,...,...,...,...,...,...,...
306,9.1,0.795,0.00,2.6,0.096,11.0,26.0,0.99940,3.35,0.83,9.4
360,7.1,0.735,0.16,1.9,0.100,15.0,77.0,0.99660,3.27,0.64,9.3
101,8.4,0.620,0.09,2.2,0.084,11.0,108.0,0.99640,3.15,0.66,9.8
200,8.4,0.635,0.36,2.0,0.089,15.0,55.0,0.99745,3.31,0.57,10.4


In [15]:
X_test

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
1287,6.5,0.53,0.06,2.0,0.063,29.0,44.0,0.99489,3.38,0.83,10.30
472,11.2,0.66,0.24,2.5,0.085,16.0,53.0,0.99930,3.06,0.72,11.00
520,12.7,0.59,0.45,2.3,0.082,11.0,22.0,1.00000,3.00,0.70,9.30
97,6.2,0.63,0.31,1.7,0.088,15.0,64.0,0.99690,3.46,0.79,9.30
111,8.0,0.71,0.00,2.6,0.080,11.0,34.0,0.99760,3.44,0.53,9.50
...,...,...,...,...,...,...,...,...,...,...,...
1213,7.6,0.43,0.31,2.1,0.069,13.0,74.0,0.99580,3.26,0.54,9.90
737,8.2,0.26,0.34,2.5,0.073,16.0,47.0,0.99594,3.40,0.78,11.30
285,13.4,0.27,0.62,2.6,0.082,6.0,21.0,1.00020,3.16,0.67,9.70
1265,6.2,0.65,0.06,1.6,0.050,6.0,18.0,0.99348,3.57,0.54,11.95


In [16]:
y_train

1311    6
1132    6
178     5
1009    6
595     5
       ..
306     6
360     5
101     5
200     4
132     6
Name: quality, Length: 1087, dtype: int64

In [17]:
y_test

1287    6
472     6
520     6
97      5
111     5
       ..
1213    6
737     7
285     6
1265    5
75      5
Name: quality, Length: 272, dtype: int64

### 2. KNN regression
- Implement naive K nearest neighbour regression as a function only using python and numpy. The signature of the function should be:
```python
def knn_regression(x_test, x_train, y_train, k=20):
        """Return prediction with knn regression."""
        .
        .
        .
        return y_pred
```
- Use Euclidean distance as a measure of distance.
- Make prediction with k=20 for the test set using the training data.
- Plot the true and the predicted values from the test set on a scatterplot.

-----

### 3. Weighted KNN regression
- Modify the knn_regression function by adding a weight to each neighbor that is inversely proportional to the distance.
```python
def knn_weighted_regression(x_test,x_train,y_train,k=20):
    """Return prediction with weighted knn regression."""
    ...
    return y_pred
```
- Make prediction with k=20 for the test set using the training data.
- Plot the true and the predicted values from the test set on a scatterplot.

-----

### 4. KNN classification
- Implement the K-nearest neighbors classification algorithm using only pure Python3 and numpy! Use L2 distance to find the neighbors. The prediction for each class should be the number of neighbors supporting the given class divided by k (for example if k is 5 and we have 3 neighbors for class A, 2 for class B and 0 for class C neighbors, then the prediction for class A should be 3/5, for class B 2/5, for class C 0/5). Use the one-hot encoded labels!
```python
def knn_classifier(X_train, y_train, X_test, k=20):
  """Return prediction with knn classification."""
    ...
    return y_pred
```

- Make prediction with k=20 for the test set using the training data.

-----

### 5. Compare the models
- Make a baseline model: this can be the mean value of the training labels for every sample.
- Compare the regression and classification models to the baseline: You can do this by rounding the continous predictions of the regression to the nearest integer. Calculate the accuracy (fraction of correctly classified samples) of the models.
- Check your KNN implementations by running the sklearn built-in model. 
You can run it for any model you implented. The predictions should be the same as yours. Some help:
  ```python
  from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
  knn= KNeighborsRegressor(20, weights="distance")
  #knn= KNeighborsClassifier(20, weights="uniform")
  knn.fit(X_train, y_train)
  knn.predict(X_test)
  ```
- Write down your observations.
----

### Hints:
- On total you can get 10 points for fully completing all tasks.
- Decorate your notebook with questions, explanation etc, make it self contained and understandable!
- Comment your code when necessary!
- Write functions for repetitive tasks!
- Use the pandas package for data loading and handling
- Use matplotlib and seaborn for plotting or bokeh and plotly for interactive investigation
- Use the scikit learn package for almost everything
- Use for loops only if it is really necessary!
- Code sharing is not allowed between students! Sharing code will result in zero points.
- If you use code found on web, it is OK, but, make its source clear!